In [1]:
# Necessary Import Statements
import pandas as pd
import numpy as np

# Datasets converted to Dataframes
#BetaData = pd.read_csv('~/ASDRP/Data/BetaData.csv', low_memory=False)
#data = pd.read_csv('~/ASDRP/Data/GammaData.csv', low_memory=False)
data = pd.read_csv('PS_2021.01.18_09.15.37.csv', low_memory=False)

# Initializing dataframes
duplicates = pd.DataFrame(columns=list(data.columns))
deDupedData = pd.DataFrame(columns=list(data.columns))

# Removing entries with no dates
data.dropna(how='any', subset=['pl_pubdate'], inplace=True)

# Removing entries with bad dates
for index, row in data.iterrows():
    date = data['pl_pubdate'][index]
    try:
        if len(date) < 8:
            data['pl_pubdate'][index] = date + '-01'
        data['pl_pubdate'][index] = pd.to_datetime(data['pl_pubdate'][index]) # This line is redundant, remove either this or line 56
    except:
        data.drop([index], inplace=True)
        data.reset_index(drop=True)

# Converting dates to datetime objects
data['pl_pubdate'] = pd.to_datetime(data['pl_pubdate'])

# Sorting the data
sortedData = data.sort_values(by=['pl_name', 'pl_pubdate'], ascending= [1,0], inplace=False, na_position='last')

# Store the duplicates before removing them
duplicates = pd.DataFrame(columns=list(data.columns))
mask = sortedData.duplicated(subset='pl_name', keep='first')
df_keep = sortedData.loc[~mask]
duplicates = duplicates.append(sortedData.loc[mask])

# Dropping the duplicates
deDupedData = sortedData.drop_duplicates(subset = 'pl_name', keep='first', inplace=False, ignore_index=True)

# Printing the first 10 entries of the DeDupedData
print(deDupedData[['pl_name', 'pl_pubdate']][0:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                   pl_name pl_pubdate
0                 11 Com b 2011-08-01
1                 11 UMi b 2017-03-01
2                 14 And b 2011-08-01
3                 14 Her b 2017-03-01
4               16 Cyg B b 2017-03-01
5                 18 Del b 2011-08-01
6  1RXS J160929.1-210524 b 2015-03-01
7                 24 Boo b 2018-08-01
8                 24 Sex b 2011-01-01
9                 24 Sex c 2011-01-01


In [2]:

# Imports the datasets
phl = pd.read_csv('PHL_Dataset.csv', low_memory=False)


In [3]:
deDupedData.head()

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292
0,2,11 Com b,11 Com,b,HD 107383,HIP 60202,NaN,NaN,0,2,1,0,0,Radial Velocity,2007,<a refstr=LIU_ET_AL__2008 href=https://ui.adsa...,2008-01,Ground,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,1,0,0,0,0,0,0,0,0,0,Published Confirmed,0,<a refstr=KUNITOMO_ET_AL__2011 href=https://ui...,NaN,NaN,NaN,NaN,1.21,0.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-07-23,2011-08-01,2014-07-23,2,1,2,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,3,11 UMi b,11 UMi,b,HD 136726,HIP 74793,NaN,NaN,1,1,1,0,0,Radial Velocity,2009,<a refstr=DOLLINGER_ET_AL__2009 href=https://u...,2009-10,Ground,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,1,0,0,0,0,0,0,0,0,0,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,516.21997,3.2,-3.2,0,1.53,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-04,2017-03-01,2018-09-06,0,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,6,14 And b,14 And,b,HD 221345,HIP 116076,NaN,NaN,0,1,1,0,0,Radial Velocity,2008,<a refstr=SATO_ET_AL__2008 href=https://ui.ads...,2008-12,Ground,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,1,0,0,0,0,0,0,0,0,0,Published Confirmed,0,<a refstr=KUNITOMO_ET_AL__2011 href=https://ui...,NaN,NaN,NaN,NaN,0.68,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-07-23,2011-08-01,2014-07-23,0,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,12,14 Her b,14 Her,b,HD 145675,HIP 79248,NaN,NaN,1,1,1,0,0,Radial Velocity,2002,<a refstr=BUTLER_ET_AL__2003 href=https://ui.a...,2003-01,Ground,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1,0,0,0,0,0,0,0,0,0,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,1773.40002,2.5,-2.5,0,2.93,0.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-04,2017-03-01,2018-09-06,0,1,4,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,17,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,NaN,NaN,1,3,1,0,0,Radial Velocity,1996,<a refstr=COCHRAN_ET_AL__1997 href=https://ui....,1997-07,Ground,Multiple Observatories,Multiple Telescopes,Multiple Instruments,1,0,0,0,0,0,0,0,0,0,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,798.5,1,-1,0,1.66,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-04,2017-03-01,2018-09-06,5,1,4,3,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Removes any planets that have an NA eccentricity value
deDupedData = deDupedData[deDupedData['pl_orbeccen'].notna()]
deDupedData = deDupedData[deDupedData['pl_rade'].notna()]

# Resets the data row index
deDupedData = deDupedData.reset_index (drop = True)

In [5]:
deDupedData['st_spectype']

0      M8.5
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
490     NaN
491     NaN
492      F5
493    G0 V
494    G0 V
Name: st_spectype, Length: 495, dtype: object

In [6]:
# Makes an empty k_type dataframe with same column headers
k_type = pd.DataFrame(columns=list(data.columns))

# Makes an empty k_type_habitable dataframe to store planets that result out of this code
k_type_habitable = pd.DataFrame(columns=list(data.columns))

# Fills the k_type dataframe with K-type host stars' planets
k_type = deDupedData.loc[(deDupedData['st_spectype'].str.startswith('K', na=False))]

# Resets the k_type row index
k_type = k_type.reset_index (drop = True)

In [7]:
# Takes out planets that have no planet radius and eccentricity
phl = phl[phl['pl_rade'].notna()]
phl = phl[phl['pl_orbeccen'].notna()]

# Resets index
phl = phl.reset_index (drop = True)

In [8]:
# Makes a new empty dataframe for the m-type planets in the PHL dataset
phl_k_type = pd.DataFrame(columns=list(phl.columns))

In [9]:
# Fills the empty dataframe with m-type planets from the PHL dataset
phl_k_type = phl.loc[(phl['st_spectype'].str.startswith('K', na=False))]

# Resets index
phl_k_type = phl_k_type.reset_index (drop = True)

In [10]:
def check_match(planet, control_planet):
    if pd.isna(planet['pl_dens']):
        if (not pd.isna(planet['pl_radj'])) & (not pd.isna(planet['pl_massj'])):
            planet['pl_dens'] = (planet['pl_massj'] * (1.898 * (10 ** 27) * 1000)) \
            / (4 * np.pi * ((planet['pl_radj'] * 43441 * 160934) ** 3) / 3)
        else:
            return
        
    if pd.isna(planet['pl_orbsmax']) and (pd.isna(planet['pl_ratdor']) or pd.isna(planet['st_rad'])):
        return

    # Calculates orbital period in days for main planet
    if pd.isna(planet['pl_orbper']):
        if pd.isna(planet['pl_ratdor']):
            planet['pl_orbper'] = np.sqrt(planet['pl_orbsmax']**3) * 365
        elif not pd.isna(planet['st_rad']): 
            planet['pl_orbper'] = np.sqrt((planet['pl_ratdor']*planet['st_rad'])**3) * 365
        if pd.isna(planet['pl_orbper']):
            return
    
    if pd.isna(control_planet['pl_orbsmax']) and (pd.isna(control_planet['pl_ratdor']) or pd.isna(control_planet['st_rad'])):
        return

    # Calculates orbital period in days for phl planet
    if pd.isna(control_planet['pl_orbper']):
        if pd.isna(control_planet['pl_ratdor']):
            control_planet['pl_orbper'] = np.sqrt(control_planet['pl_orbsmax']**3) * 365
        elif not pd.isna(control_planet['st_rad']): 
            control_planet['pl_orbper'] = np.sqrt((control_planet['pl_ratdor']*control_planet['st_rad'])**3) * 365
        if pd.isna(control_planet['pl_orbper']):
            return
    
    if pd.isna(planet['pl_orbeccen']):
        return
    
    

    if (float(planet['pl_dens']) >= 4) & (float(planet['pl_dens']) <= 7):
        if (planet['pl_controv_flag'] == '0'):
            if ('Transit' in planet['discoverymethod']):
                if (planet['pl_orbeccen'] <= 0.2):
                    if (np.abs(float(planet['pl_orbper']) - float(control_planet['pl_orbper'])) <= 50):
                        global k_type_habitable
                        k_type_habitable = k_type_habitable.append(planet, ignore_index = True)


In [11]:
for i in range (len(k_type['pl_rade'])): 
    current_dist = np.abs(phl_k_type['pl_rade'][0]-k_type['pl_rade'][i])
    if current_dist < 20:
        planet = k_type.loc[i].copy()
        control_planet = phl_k_type.loc[0].copy()
        check_match(planet, control_planet)

In [12]:
k_type_habitable

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292
0,383,GJ 143 b,GJ 143,b,HD 21749,HIP 16069,NaN,NaN,1,1,2,0,0,Transit,2019,<a refstr=TRIFONOV_ET_AL__2019 href=https://ui...,2019-02,Space,Transiting Exoplanet Survey Satellite (TESS),0.1 m TESS Telescope,TESS CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=DRAGOMIR_ET_AL__2019 href=https://ui...,35.61253,0.0006,-0.00062,0,0.1915,0.0058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-22,2019-04-01,2019-04-25,1,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
k_type_habitable['pl_orbper']

0    35.61253
1    13.86375
Name: pl_orbper, dtype: object

In [ ]:
phl_k_type

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289
0,23029,Kepler-61 b,Kepler-61,b,NaN,NaN,TIC 271537530,Gaia DR2 2077889892023816960,1,1,1,0,0,Transit,2013,<a refstr=BALLARD_ET_AL__2013 href=https://ui....,2013-08,Space,Kepler,0.95 m Kepler Telescope,Kepler CCD Array,0,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=BALLARD_ET_AL__2013 href=https://ui....,59.87756,0.0002,-0.0002,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,12.184,0.023,-0.023,12.248,0.022,-0.022,12.425,0.29,-0.29,9.026,NaN,NaN,15.0426,0.001305,-0.001305,NaN,NaN,NaN,14.2406,0.0063,-0.0063,14.995,NaN,NaN,2014-05-14,2013-08,2014-05-14,1,0,0,0,0,2,NaN,NaN,NaN


In [ ]:
phl_k_type['pl_orbper']

0    59.87756
Name: pl_orbper, dtype: object